In [ ]:
#ignore
import random
import numpy as np
import pandas as pd
import panel as pn
import holoviews as hv

pn.extension('vtk')

We are very pleased to announce the 0.7.0 release of Panel which brings a ton of new features, enhancements and many important bug fixes. Many thanks to the 20 contributors to this release (listed at the bottom). This release introduced only minimal changes in existing APIs, indicating that Panel is progressing towards a more stable phase of development. One of the major goals in this release was better compatibility with the Jupyter ecosystem, which culminated in the ipywidgets support. The next major release will be the 1.0 release, which will involve some minor API cleanup and a number of long anticipated features, including a number of polished inbuilt templates and the ability to serve existing Jupyter widgets as part of a Panel app.  

### **Major features**:

* Ability to render any Panel output as an ipywidget model (#745, #755, #771)
* Added `.jscallback` method to Viewable objects (#665)

### **New components**:

#### Widgets

* Added new ``Progress`` bar widget (#726)
* Added new ``DataFrame`` editing widget (#767)
* Added new ``PasswordInput`` widget (#655)
* Added new ``TextAreaInput`` widget (#658)

#### Panes

* Added new ``DataFrame`` pane with the ability to render pandas, dask and streamz dataframes (#560, #751)
* Added new ``Streamz`` pane (#767, #769)
* Added new ``VTKVolume`` pane (#605, #606, #715, #729)
* Added new ``Video`` pane (#696)

#### Layouts

* Added new `GridBox` layout (#608, #761, #763)
* Added new `Divider` component (#756)

### **Enhancements**:

* Major improvements to the `Pipeline` object to allow branching graphs (#712, #735, #737, #770)
* Ability to bi-directionally link objects in Javascript using `.jslink` (#764)
* Make Row/Column scrollable (#760)
* Added repr and a kill_all_servers method to pn.io.state.state (#697)
* Templates can now render inside a notebook cell (#666)
* WidgetBox can be disabled programmatically (#532)

### **API Changes**:

* The ``Audio`` widget has been deprecated in favor of a `Audio` pane. (#696)


<hr>

If you are using [Anaconda](https://www.anaconda.com/downloads), you can get latest Panel with ``conda install -c pyviz panel`` , and using pip you can install it with ``pip install panel``.

<hr>

## ipywidget support

Panel is built on top of bokeh which ships with its own standalone server and has always provided integration in Jupyter. Panel itself has relied on some custom extensions for Jupyter support which don't necessarily work in some non-standard notebook and Jupyter environments such as the recently released [Voilà](https://github.com/voila-dashboards/voila) dashboard server. After working with the Jupyter and Bokeh developers we have now released the `jupyter_bokeh` library and extension which allows displaying Bokeh and Panel models as ipywidgets and therefore ensures that bi-directional communication works in any environment that supports the Jupyter widget protocol.

In Panel we can enable this globally using **`pn.extension(comm='ipywidgets')`** or by explicitly converting objects to an ipywidget using **`pn.ipywidget(obj)`**.

In [ ]:
import ipywidgets as ipw

accordion = ipw.Accordion(children=[
    pn.ipywidget(pn.Column(
        pn.widgets.FloatSlider(),
        pn.widgets.TextInput()
    )),
    pn.ipywidget(hv.Curve([1, 2, 3])),
    pn.ipywidget(hv.Area([1, 2, 3]).opts(responsive=True, min_height=300))
])

accordion.set_title(0, 'Widgets')
accordion.set_title(1, 'Curve')
accordion.set_title(2, 'Area')

<center><img src="./images/accordion.png" width="80%"></center>

## Support for .jscallback and improved .jslink

Panel has long had support for linking the parameters of two objects in Javascript using the `.jslink` method. In  this release `.jslink` can now be invoked bi-directionally:

In [ ]:
kwargs = dict(start=0, end=1, step=0.1, align='center')
slider = pn.widgets.FloatSlider(name='Slider', **kwargs)
spinner = pn.widgets.Spinner(name='Spinner', **kwargs)

slider.jslink(spinner, value='value', bidirectional=True)

pn.Row(slider, spinner)

Additionally the `.jscallback` method has been added to generate arbitrary callbacks in response to some change to a property:

In [ ]:
value1 =   pn.widgets.Spinner(value=0, width=75)
operator = pn.widgets.Select(value='*', options=['*', '+'], width=50, align='center')
value2 =   pn.widgets.Spinner(value=0, width=75)
button =   pn.widgets.Button(name='=', width=50)
result =   pn.widgets.StaticText(value='0', width=50, align='center')

button.jscallback(clicks="""
if (op.value == '*') 
  result.text = (v1.value * v2.value).toString()
else
  result.text = (v1.value + v2.value).toString()
""", args={'op': operator, 'result': result, 'v1': value1, 'v2': value2})

pn.Row(value1, operator, value2, button, result)

## Improved Pipelines

Previously the [``Pipeline`` class](https://panel.pyviz.org/user_guide/Pipelines.html) allowed setting up linear pipelines to implement a multi-stage workflow. The ``Pipeline`` class was completely overhauled in this release to make it easy to lay out the individual components yourself and most importantly to set up an arbitrary graph of pipeline stages, e.g. pipelines now allow diverging and converging branches for more flexible workflows than before. Below is the definition and the overview of a complex graph based pipeline with diverging and converging stages:

```python
dag = pn.pipeline.Pipeline()

dag.add_stage('Input', Input)
dag.add_stage('Multiply', Multiply)
dag.add_stage('Add', Add)
dag.add_stage('Result', Result)
dag.add_stage('Export', Export)

dag.define_graph({'Input': ('Multiply', 'Add'), 'Multiply': 'Result', 'Add': 'Result', 'Result': 'Export'})
```

<center><img src="./images/pipelines.png" style="margin: auto" width="90%"></center>

## Improved Templates

Since Panel 0.6.0 it has been possible to declare custom [Templates](https://panel.pyviz.org/user_guide/Templates.html) to take full control over the layout and visual styling of the application or dashboard. In this release we now support rendering custom templates in a notebook and even declaring separate templates for notebook and server usage. In the next release we will focus on providing a number of custom templates built on common JS/CSS frameworks such as Materialize UI, GridStack, and reveal.js.

<center><img src="./images/materialize.png" style="margin: auto" width="90%"></center>

## New Components

This release includes a variety of new components contributing to the growing set of widgets, panes and layouts showcased in the [reference gallery](https://panel.pyviz.org/reference/index.html).

#### Progress bars

The [``Progress`` widget](http://panel.pyviz.org/reference/widgets/Progress.html) displays the progress towards some target based on the current `value` and the `max` value. If no `value` is set the ``Progress`` widget is in indeterminate mode and will animate depending on the ``active`` parameter state.

In [ ]:
#ignore
running = pn.Column(*[
    pn.Row(pn.panel(bs, width=100), pn.widgets.misc.Progress(width=300, value=10+i*10, bar_color=bs), pn.widgets.misc.Progress(width=300, bar_color=bs))
        for i, bs in enumerate(pn.widgets.misc.Progress.param.bar_color.objects)
])
running

#### DataFrame widget
 
The [``DataFrame`` widget](http://panel.pyviz.org/reference/widgets/DataFrame.html) allows editing an existing pandas DataFrame using a custom `DataTable`.

In [ ]:
import pandas as pd

df = pd.DataFrame({'int': [1, 2, 3], 'float': [3.14, 6.28, 9.42], 'str': ['A', 'B', 'C']}, index=[1, 2, 3])

pn.widgets.DataFrame(df, widths={'index': 10, 'int': 10, 'float': 50, 'str': 100}, width=200)

#### PasswordInput & TextAreaInput widgets

New [``PasswordInput``](http://panel.pyviz.org/reference/widgets/PasswordInput.html) and [``TextAreaInput``](http://panel.pyviz.org/reference/widgets/TextAreaInput.html) make it possible to enter hidden text and provide multi-line text inputs to Panel:

In [ ]:
#ignore
password_input = pn.widgets.PasswordInput(name='Password Input', placeholder='Enter a string here...')
text_area = pn.widgets.TextAreaInput(name='Text Area Input', placeholder='Enter a string here...', height=100)

pn.Row(password_input, text_area)

#### DataFrame Pane

The [`DataFrame` pane](http://panel.pyviz.org/reference/panes/DataFrame.html) renders pandas, dask and streamz dataframes while exposing a range of options to control the formatting.

<img src="./images/streamz_dataframe.gif"></img>

#### Streamz Pane

The Streamz pane accepts any [streamz](https://github.com/python-streamz/streamz) Stream to allow streaming arbitrary objects, e.g. the [basic example](https://panel.pyviz.org/reference/panes/Streamz.html) in the documentation demonstrates how to quickly put together a streaming vega plot:

<center><img src="./images/streamz_vega.gif" width="40%"></center>

#### Video Pane

The [``Video`` pane](https://panel.pyviz.org/reference/pane/Video.html)  uses a standard HTML5 media player widget to display any mp4, webm or ogg video file. Like the corresponding ``Audio`` pane the current timestamp, volume and play state can be toggled from Python and Javascript:

In [ ]:
video = pn.pane.Video('https://sample-videos.com/video123/mp4/720/big_buck_bunny_720p_1mb.mp4',
                      width=640, height=480)

video

#### VTKVolume

The [VTKVolume pane](https://pyviz-dev.github.io/panel/reference/panes/VTKVolume.html) uses the vtk.js library to render interactive, volumetric plots with control over opacity and the color curve.

<center><img src="./images/vtkjs.gif" style="margin: auto;" style="margin: auto" width="90%"></img></center>

#### GridBox layout

The [``GridBox`` layout](https://panel.pyviz.org/reference/layouts/GridBox.html) compliments the existing `Row`, `Column`, `Tabs` and `GridSpec` layouts in that it allows wrapping the list of items provided to it by the desired number of rows or columns:

In [ ]:
rcolor = lambda: "#%06x" % random.randint(0, 0xFFFFFF)

box = pn.GridBox(*[pn.pane.HTML(background=rcolor(), width=50, height=50) for i in range(22)], ncols=4)

In [ ]:
#ignore
slider = pn.widgets.IntSlider(name='ncols', start=4, end=8)
slider.link(box, value='ncols')

pn.Column(slider, box).embed(max_opts=6)

#### Divider

The new ``Divider`` component also nicely complements the existing `Spacer` components making it easy to draw a visual divider between vertically stacked components.

In [ ]:
pn.Column(
    pn.layout.Divider(),
    pn.Row(pn.layout.HSpacer(), '# Title', pn.layout.HSpacer()),
    pn.layout.Divider()
)

## Contributors

Many thanks to the many contributors to this release:
    
* Philipp Rudiger (@philippjfr): Maintainer & lead developer
* Xavier Artusi (@xavArtley): VTK support
* James A. Bednar (@jbednar): Documentation
* Andrew Tolmie (@DancingQuanta): FileInput widget
* Arne Recknagel (@a-recknagel): Python 3.8 support, build improvements
* Julius Winkelmann (@julwin): TextAreaInput, PasswordInput
* Pav A (@rs2): Example notebooks
* Ed Jung (@xtaje): Default values fix
* Karthick Perumal (@Karamya): Audio widget enhancements
* Christopher Ball (@ceball): Build and doc improvements
* Andrew Huang (@ahuang11): Disabling widget boxes
* Eduardo Gonzalez (@eddienko): Fixing Django docs
* Jacob Barhak (@Jacob-Barhak): Updated Markdown docs
* Jean-Luc Stevens (@jstevens): Cross-selector fixes
* Julia Signell (@jsignell): Documentation fixes
* Landung "Don" Setiawan (@lsetiawan): StoppableThread improvements
* Mateusz Paprocki (@mattpap): Build infrastructure
* Maxime Borry (@maxibor): Widget fixes
* Stefan Farmbauer (@RedBeardCode): File-like object support on images
* @kleavor: Fixed GridSpec override behavior